In [76]:
# импортирование библиотек
import pandas as pd
import seaborn as sns
import numpy as np
import os
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.metrics import accuracy_score

In [77]:
data = pd.read_csv('train_data.csv')

In [78]:
data.drop('Unnamed: 0', axis = 1, inplace = True)
data.head()

,cost,level_1_1,level_1_2,level_1_3,level_2_1,level_2_2,region,is_44,title_vector_0,title_vector_1,...,title_vector_11,title_vector_12,title_vector_13,title_vector_14,title_vector_15,title_vector_16,title_vector_17,title_vector_18,title_vector_19,supplier_id
0,6710121.0,1,0,0,0,0,60,1,0.321117,-0.317464,...,-0.304298,-0.168377,0.104410,0.252260,0.154310,-0.331585,0.287732,-0.602622,-0.652956,44
1,5395.0,0,0,1,0,1,51,1,0.079101,0.098359,...,0.004997,0.353382,0.356620,0.149338,-0.058060,0.163763,-0.139090,-0.152761,-0.176053,9
2,34330.0,0,0,1,0,1,9,0,0.083967,-0.029964,...,-0.168681,-0.023522,-0.110543,0.171777,0.005388,0.211797,-0.049800,-0.231361,0.127630,52
3,1200000.0,1,0,0,0,0,46,1,0.130799,0.020342,...,-0.162256,0.151090,-0.113475,0.215033,0.280156,0.271204,-0.001403,-0.094558,-0.171027,5
4,1144000.0,1,0,0,0,0,45,1,-0.019896,-0.080144,...,-0.143452,0.270669,-0.095452,0.190687,0.180496,0.242237,0.012682,-0.176461,-0.009448,143


In [79]:
X = data.drop('supplier_id', axis = 1)
y = data['supplier_id']

In [80]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

In [81]:
from catboost import CatBoostClassifier

In [82]:
model = CatBoostClassifier(
    loss_function='MultiClass',  # Use 'MultiClass' for multiclass classification
    verbose=False,  # Set to True to see training logs
    random_state=42,)
model.fit(
        x_train, y_train,
        eval_set=(x_test, y_test),
        early_stopping_rounds=10,
        verbose=False,
        
    )

In [71]:
y_pred = model.predict(x_test)

In [72]:
# точность модели
from sklearn.metrics import accuracy_score
accuracy_score(y_pred, y_test)

0.2725540025412961

In [73]:
def score(x_test, y_test, model):
    count = 0
    for i in range(len(x_test)):
        x = pd.Series(model.predict_proba(x_test.values[i])).sort_values(ascending=False).head(5).index
        if y_test.values[i] in x:
            count += 1
    return count/len(y_test)

In [74]:
# точность того, что поставщик из тестовой выборки попадет в топ 5 рекомендованных поставщиков 
score(x_test, y_test, model)

0.5546378653113088

In [11]:
# сохранение модели 
model.save_model('rec_model')